<a href="https://colab.research.google.com/github/Brymer-Meneses/PyTorch-Basics/blob/master/PyTorch_CatsVsDogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites

In [ ]:
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

In [2]:
from zipfile import ZipFile

In [3]:
zipPath = '/content/kagglecatsanddogs_3367a.zip'
zip = ZipFile(zipPath, 'r')
zip.extractall()

In [7]:
import tensorflow as tf

In [8]:
import os

num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

Deleted 1590 images


# Modules

In [47]:
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms



In [36]:
path = '/content/PetImages'
dataset = ImageFolder(path)

trainData, testData, trainLabel, testLabel = train_test_split(dataset.imgs, dataset.targets, test_size =0.2, random_state = 0)
transform = transforms.Compose([
  transforms.Resize((200,200)),
  transforms.ToTensor(),
  transforms.Normalize([0.5]*3, [0.5]*3)
])

In [ ]:
class ImageLoader(Dataset):
  def __init__(self, dataset, transform=None):
    self.dataset = dataset
    self.transform = transform

  def __getitem__(self, item):
    image = Image.open(self.dataset[item][0])
    if transform is not None:
      return self.transform(image), self.dataset[item][1]
    return image, self.dataset[item][1]
  
  def __len__(self):
    return len(self.dataset)
  
  def checkChannel(self, dataset):
    datasetRGB = []
    for index in range(len(dataset)):
      if Image.open(dataset[index][0]).getbands() == ('R', 'G', 'B'):
        datasetRGB.append(dataset[index])
    return datasetRGB
  
  def getResizedImage(self, item):
    image = Image.open(self.dataset[item][0])
    _, _, width, height = image.getbbox()

    factor = (0,0, width, width) if width > height else (0,0,height, height)

    return image.crop(factor)


imageLoader = ImageLoader(trainData, transform)

dataLoader = DataLoader(imageLoader, batch_size = 10, shuffle = True)
data= iter(dataLoader)
d = next(data)
print(d)

In [54]:
from torch.nn import Module, Conv2d, Linear, Flatten, MaxPool2d
from torch.nn.functional import relu

In [53]:
class Network(Module):
  def __init__(self):
    super(Network, self).__init__()
    self.conv_1 = Conv2d(in_channels = 3, out_channels = 64, kernel_size = 5)
    self.conv_2 = Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3)
    self.conv_3 = Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3)

    self.maxPooling = MaxPool2d(kernel_size =4)

    self.fc1 = Linear(in_features = 256, out_features = 128)
    self.fc2 = Linear(in_features = 128, out_features =64)
    self.out = Linear(in_features = 64, out_features = 2)
  
  def forward(self):
    x = self.conv_1(x)
    x = self.maxPooling(x)
    x = relu(x)

    x = self.conv_2(x)
    x = self.maxPooling(x)
    x = relu(x)

    x = self.conv_3(x)
    x = self.maxPooling(x)
    x = relu(x)

    x = self.fc1(x)
    x = relu(x)

    x = self.fc2(x)
    x = relu(x)

    return self.out(x)